In [1]:
import pandas as pd

data = pd.read_csv('../Dataset/SafeBite_preprocessed_data_loo_enc_price.csv')

(data.head())


,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Price ($),Customer rating,Is_Allergen
0,6.170000,6.170,12.343736,12.471786,13.408,6.170000,10.15,3.1,1
1,10.150000,10.150,12.387473,12.519167,13.806,10.150000,6.17,4.5,1
2,12.500000,17.480,12.378423,12.031228,11.584,17.480000,19.65,4.1,1
3,13.223333,19.650,12.386201,12.069298,11.801,19.650000,17.48,4.7,1
4,12.402356,17.925,12.410036,12.185965,12.466,12.072073,10.83,3.7,1


In [2]:
unseen_data = pd.read_csv("../Dataset/sample.csv")
unseen_data.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Price ($),Customer rating
0,Blueberry Protein Bar,Blueberries,Honey,Almond oil,Vanilla extract,Tree nuts,2.99,4.2
1,Mango Chia Pudding,Mango,Coconut sugar,Coconut milk,Cardamom,Coconut,4.49,4.5
2,Beetroot Chips,Beetroot,NaN,Olive oil,Sea salt,NaN,3.25,3.8
3,Pumpkin Spice Muffin,Pumpkin,Maple syrup,Butter,"Cinnamon, Nutmeg","Wheat, Dairy",5.75,4.7
4,Lentil Soup,Lentils,NaN,Olive oil,Herbs de Provence,NaN,4.00,4.0


In [3]:
# Load the pre-trained encoder and model:
import joblib

loaded_encoder = joblib.load('../Models/loo_encoder.pkl')
loaded_model = joblib.load('../Models/best_xgboost_model.pkl')
print("Model and encoder loaded successfully!")

Model and encoder loaded successfully!


In [4]:
example_data = pd.DataFrame({
    'Food Product': ['Gluten-Free Chocolate Cake', 'Chocolate Cake'],
    'Main Ingredient': ['Almond flour', 'Wheat flour'],
    'Sweetener': ['Coconut sugar', 'Sugar'],
    'Fat/Oil': ['Coconut oil', 'Butter'],
    'Seasoning': ['Vanilla extract, Cocoa', 'Vanilla extract, Cocoa'],
    'Allergens': ['None', 'Wheat-Dairy'],
    'Price ($)': [7.50, 7.55],
    'Customer rating': [4.5, 4.6]
})

In [5]:
training_features = loaded_model.feature_names_in_
training_features

array(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil',
       'Seasoning', 'Allergens', 'Price ($)', 'Customer rating'],
      dtype='<U15')

In [6]:
categorical_cols = ['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning', 'Allergens']


In [7]:
data_encoded = loaded_encoder.transform(example_data[categorical_cols])
data_encoded

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens
0,12.402356,12.402356,12.402356,12.402356,12.402356,12.165608
1,12.402356,16.355000,12.319891,12.444471,12.402356,12.402356


In [8]:
encoded_cols = data_encoded.columns
encoded_cols

Index(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning',
       'Allergens'],
      dtype='object')

In [9]:
example_data = pd.concat([example_data[['Price ($)', 'Customer rating']], data_encoded], axis=1)
example_data
     

,Price ($),Customer rating,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens
0,7.50,4.5,12.402356,12.402356,12.402356,12.402356,12.402356,12.165608
1,7.55,4.6,12.402356,16.355000,12.319891,12.444471,12.402356,12.402356


In [10]:
example_data = example_data[training_features]
example_data

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Price ($),Customer rating
0,12.402356,12.402356,12.402356,12.402356,12.402356,12.165608,7.50,4.5
1,12.402356,16.355000,12.319891,12.444471,12.402356,12.402356,7.55,4.6


In [11]:
training_features


array(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil',
       'Seasoning', 'Allergens', 'Price ($)', 'Customer rating'],
      dtype='<U15')

In [12]:

# Making predictions using the loaded model
predictions = loaded_model.predict(example_data)

# Displaying predictions with descriptive output
for i, prediction in enumerate(predictions):
    result = "No allergens detected" if prediction == 0 else "Allergens present"
    print(f"Prediction for example {i + 1}: {result} ")


Prediction for example 1: No allergens detected 
Prediction for example 2: Allergens present 


In [13]:
print("Columns in unseen_data:", unseen_data.columns)


Columns in unseen_data: Index(['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning',
       'Allergens', 'Price ($)', 'Customer rating '],
      dtype='object')


In [14]:
unseen_data.columns = unseen_data.columns.str.strip()


In [19]:
import joblib
import pandas as pd

# Load the saved model and encoder
model = joblib.load("../Models/best_xgboost_model.pkl")
encoder = joblib.load("../Models/loo_encoder.pkl")

# Load the unseen data
unseen_data = pd.read_csv("../Dataset/sample.csv")

# Strip column names of whitespace
unseen_data.columns = unseen_data.columns.str.strip()

# Preprocess unseen data (apply the encoder for categorical columns)
categorical_cols = ['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning', 'Allergens']
unseen_data_encoded = encoder.transform(unseen_data[categorical_cols])

# Combine encoded categorical features with numerical columns
numerical_cols = ['Price ($)', 'Customer rating']
if 'Customer rating' not in unseen_data.columns:
    raise KeyError("The column 'Customer rating' is missing in the dataset.")

unseen_data_processed = pd.concat([unseen_data_encoded, unseen_data[numerical_cols]], axis=1)

# Check for missing features
training_features = model.feature_names_in_
missing_features = set(training_features) - set(unseen_data_processed.columns)
if missing_features:
    print(f"Missing features: {missing_features}")
    unseen_data_processed['Customer rating'] = unseen_data['Customer rating']  # Ensure the column is included

# Ensure the columns match the training features
unseen_data_processed = unseen_data_processed[training_features]

# Make predictions
predictions = model.predict(unseen_data_processed)

# Save predictions to CSV
unseen_data['Is_Allergen'] = predictions
unseen_data.to_csv("../Dataset/Predictions_sample_dataset.csv", index=False)

print("Predictions have been saved to Predictions_sample_dataset.csv")


Predictions have been saved to Predictions_sample_dataset.csv


In [20]:
data = pd.read_csv('../Dataset/Predictions_sample_dataset.csv')


In [21]:
data.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Price ($),Customer rating,Is_Allergen
0,Blueberry Protein Bar,Blueberries,Honey,Almond oil,Vanilla extract,Tree nuts,2.99,4.2,1
1,Mango Chia Pudding,Mango,Coconut sugar,Coconut milk,Cardamom,Coconut,4.49,4.5,1
2,Beetroot Chips,Beetroot,NaN,Olive oil,Sea salt,NaN,3.25,3.8,1
3,Pumpkin Spice Muffin,Pumpkin,Maple syrup,Butter,"Cinnamon, Nutmeg","Wheat, Dairy",5.75,4.7,1
4,Lentil Soup,Lentils,NaN,Olive oil,Herbs de Provence,NaN,4.00,4.0,1
